In [1]:
import os
from openai import OpenAI
import openai
from tool import videoreader
# 创建一个OpenAI客户端实例
client = OpenAI(
    api_key="xai-KCpIDETeOXhYOhegEVAAHJWhbNO4qO9TuuXntFJAYOpUI9c6YyEiX2vNPCMZkubqtZnLhNN5C7Dpkowv",
    base_url="https://gateway.ai.cloudflare.com/v1/627f1b1f372e3a198dc32573bbc6f720/x-ai/grok"  # 替换为你的自定义API域
)

## Set the API key and model name
MODEL="grok-2-vision-1212"

In [2]:
dimension = 'aesthetic_quality'
from PromptTemplate4GPTeval import Prompt4AestheticQuality
prompt_template = Prompt4AestheticQuality

In [3]:
import json
# data_prepath = r'D:\Astudying\VideoEval\data4dimensions'
data_prepath = "../../data4dimensions/"
with open("./Human_anno_sora_test/{}.json".format(dimension)) as f:
    human_anno = json.load(f)

file_path ="./Grok_eval_res/{}_llmeval.json".format(dimension,dimension)

with open(file_path,'r') as f:
    s = json.load(f)



In [4]:
l1 = list(range(0,len(human_anno)))
# l2 = list(range(2,len(human_anno),3))
l = l1

In [5]:
# skip_index = list(range(0, len(human_anno),5))
model2message = {
'cogvideox5b':"12 frames from cogvideox5b,which you need to evaluate \n",
'kling':"10 frames from kling ,which you need to evaluate\n ", 
'gen3': "10 frames from gen3 ,which you need to evaluate\n",
'videocrafter2':"4 frames from videocrafter2,which you need to evaluate",
'pika':"7 frames from pika ,which you need to evaluate",
'show1':"8 frames from show1,which you need to evaluate ",
'lavie':"5 frames from lavie ,which you need to evaluate",
'sora':"10 frames from sora ,which you need to evaluate",
}
for i in l:
    s[str(i)] = {}

for key, value in model2message.items():
    modelname = key
    modelmessage = value
    for i in l:             
        # if i in skip_index:
        #     continue
        # # frames = videoreader.process_video(data_dir,human_anno[i]['videos'],4 )
        # else:
        examplemodels = [x for x in model2message.keys() if x != modelname]
        frames = videoreader.process_video(data_prepath,human_anno[i]['videos'],4)
        prompten = human_anno[i]['prompt_en']
        # question = human_anno[i]['question_en']
        # subject = human_anno[i]['subject_en']
        # scene = human_anno[i]['scene_en']
        # objet = human_anno[i]['object']
        try:
            response = client.chat.completions.create(
            model=MODEL, 
            messages=[
            {
            "role": "system", "content":
                prompt_template
                }
                ,
            {
                "role": "user", "content":[
            {"type": "text",
                "text":"According to **Important Notes** in system meassage, there are examples from other models.\n"},
            *[item for examplemodel in examplemodels for item in [
                {"type":"text","text":"This example is from model {} \n".format(examplemodels.index(examplemodel)+1)},
                {"type": "image_url", "image_url": {"url": f'data:image/jpg;base64,{frames[examplemodel][0]}', "detail": "low"}}
            ]],              
                {"type":"text","text":"These are the frames from the video you are evaluating. \n"},
                *map(lambda x: {"type": "image_url", 
                    "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}},frames[modelname]),    
                {"type":"text","text":"Assuming there are a video ' scoring 'x',provide your analysis and explanation in the output format as follows:\n"},
                {"type":"text","text": "- video: x ,because ..."
},
            ],
                }
            ],
            temperature=0,
            )
            print(response.choices[0].message.content) 
            s[str(i)][modelname] = response.choices[0].message.content.replace('\n\n','\n')
        except Exception as e:
            print(f"An error occurred: {e}")
            s[i] = 'Error'

- video: 4 ,because the video exhibits good aesthetic quality across several criteria. The **structure** of the grapes is reasonable and pleasing, with a natural arrangement that avoids discomfort. The **color use** is appropriate, with vibrant green and dark purple grapes that contrast well, enhancing visual appeal. The **composition** effectively presents the grapes, focusing on their texture and color, providing all necessary visual information. The **visual appeal** is strong, with the close-up shots creating an intimate and engaging experience, and the slight moisture on the grapes adds a tactile quality. Lastly, the **harmony** of the video is well-maintained, with the colors and arrangement contributing to a cohesive and harmonious whole. While it doesn't reach the level of excellence in all aspects, it provides a visually satisfying experience with good emotional expression and creativity.
- video: 4 ,because the video exhibits strong performance in several aspects of aesthetic

KeyboardInterrupt: 

In [6]:
import json
# 使用 json 保存字典
with open(file_path, "w") as f:
    json.dump(s, f,indent=4)

In [7]:
# with open(file_path, "r") as file:
#     data = json.load(file)

# video_path = './Human_anno/object_class.json'
# with open(video_path, "r") as file:
#     annotations = json.load(file)

# for index,key in enumerate(data.keys()):       
#     for line in data[key].split('\n'):
#         if line == '' or line[0] != '-':
#             continue
#         model_name = line.split(',')[0].split(' ')[1].split(':')[0].lower()
#         score = int(line.split(',')[0].split(' ')[2])
#         annotations[index]['gpt4o_eval'][model_name] = score
    
# with open(video_path, "w") as f:
#     json.dump(annotations, f,indent=4)